### 1. 누락 데이터 처리

- 데이터분석의 정확도 -> 분석 데이터의 품질에 의해 좌우됨.
- 누락 데이터, 중복 데이터 등 오류 수정하고 분석 목적에 맞게 변형하는 과정 필요
- 일반적으로 유효한 데이터 값이 존재하지 않는 누락 데이터 -> NaN으로 표시

#### 누락 데이터 확인

In [1]:
import seaborn as sns

df = sns.load_dataset('titanic')

df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [2]:
df.info() 
# non-null값 개수 보여줌
# RangeIndex : 각 열에 존재하는 데이터 개수

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    object  
 3   age          714 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     889 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    object  
 10  adult_male   891 non-null    bool    
 11  deck         203 non-null    category
 12  embark_town  889 non-null    object  
 13  alive        891 non-null    object  
 14  alone        891 non-null    bool    
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 80.6+ KB


In [3]:
nan_deck = df['deck'].value_counts(dropna=False)
# dropna = True : Nan값 제외 유효한 데이터 개수만 출력
nan_deck

NaN    688
C       59
B       47
D       33
E       32
A       15
F       13
G        4
Name: deck, dtype: int64

In [5]:
# isnull() : 누락데이터면 True, 유효한 데이터면 False 반환
print(df.head().isnull())
print("\n")
# notnull() : 유효한 데이터면 True, 누락 데이터면 False 반환
print(df.head().notnull())

   survived  pclass    sex    age  sibsp  parch   fare  embarked  class  \
0     False   False  False  False  False  False  False     False  False   
1     False   False  False  False  False  False  False     False  False   
2     False   False  False  False  False  False  False     False  False   
3     False   False  False  False  False  False  False     False  False   
4     False   False  False  False  False  False  False     False  False   

     who  adult_male   deck  embark_town  alive  alone  
0  False       False   True        False  False  False  
1  False       False  False        False  False  False  
2  False       False   True        False  False  False  
3  False       False  False        False  False  False  
4  False       False   True        False  False  False  


   survived  pclass   sex   age  sibsp  parch  fare  embarked  class   who  \
0      True    True  True  True   True   True  True      True   True  True   
1      True    True  True  True   True   True  Tr

In [6]:
# 누락 데이터 개수 구하기
print(df.head().isnull().sum(axis=0))

survived       0
pclass         0
sex            0
age            0
sibsp          0
parch          0
fare           0
embarked       0
class          0
who            0
adult_male     0
deck           3
embark_town    0
alive          0
alone          0
dtype: int64


In [9]:
# 누락 데이터 제거
import seaborn as sns

df = sns.load_dataset('titanic')

missing_df = df.isnull()
for col in missing_df.columns:
    missing_count = missing_df[col].value_counts()
    try:
        print(col, ':', missing_count[True])
    except:
        print(col, ':', 0)
# 'deck'열의 누락 데이터 비율이 매우 높으므로 누락 데이터를 삭제하여
# 분석에서 제외하는 것이 의미 있음.

survived : 0
pclass : 0
sex : 0
age : 177
sibsp : 0
parch : 0
fare : 0
embarked : 2
class : 0
who : 0
adult_male : 0
deck : 688
embark_town : 2
alive : 0
alone : 0


In [10]:
# NaN값이 500개 이상인 열을 모두 삭제
df_thresh = df.dropna(axis=1, thresh=500)
df_thresh.columns

Index(['survived', 'pclass', 'sex', 'age', 'sibsp', 'parch', 'fare',
       'embarked', 'class', 'who', 'adult_male', 'embark_town', 'alive',
       'alone'],
      dtype='object')

In [11]:
# 891명 중 177명의 나이 데이터 없음 
# 나이가 중요한 변수인 경우, 나이 데이터가 없는 승객의 레코드 제거하는 것이 좋음.
# how = 'any' : NaN값이 하나라도 존재하면 삭제
# how = 'all' : 모든 데이터가 NaN값일 경우에만 삭제
df_age = df.dropna(subset=['age'], how='any', axis=0)
len(df_age)

714

#### 누락 데이터 치환
- 데이터 일부가 누락되었더라도 나머지 데이터를 최대한 살려서 활용하는 것이 좋은 경우가 많음.
- 데이터의 분포와 특성을 잘 나타낼 수 있는 평균값, 최빈값 등으로 누락 데이터 치환
- fillna()

In [12]:
print(df['age'].head(10))
print("\n")

# age 열의 NaN값 다른 나이 데이터의 평균으로 대체
mean_age = df['age'].mean(axis=0)
df['age'].fillna(mean_age, inplace = True)

print(df['age'].head(10))

0    22.0
1    38.0
2    26.0
3    35.0
4    35.0
5     NaN
6    54.0
7     2.0
8    27.0
9    14.0
Name: age, dtype: float64


0    22.000000
1    38.000000
2    26.000000
3    35.000000
4    35.000000
5    29.699118
6    54.000000
7     2.000000
8    27.000000
9    14.000000
Name: age, dtype: float64


In [13]:
# embark_town 열의 NaN 데이터 출력
print(df['embark_town'][825:830])
print("\n")

# 누락값 -> 최빈값으로 대체
most_freq = df['embark_town'].value_counts(dropna=True).idxmax()
print(most_freq)
print('\n')

df['embark_town'].fillna(most_freq, inplace = True)

print(df['embark_town'][825:830])

825     Queenstown
826    Southampton
827      Cherbourg
828     Queenstown
829            NaN
Name: embark_town, dtype: object


Southampton


825     Queenstown
826    Southampton
827      Cherbourg
828     Queenstown
829    Southampton
Name: embark_town, dtype: object


- 누락 데이터가 NaN으로 표시되지 않은 경우
    - replace() 메소드 활용하여 numpy에서 지원하는 np.nan으로 변경
    - df.replace('?', np.nan, inplace = True)

In [15]:
df = sns.load_dataset('titanic')
# 데이터셋 특성 상 서로 이웃하는 데이터끼리 유사성 가질 가능성 높음
print(df['embark_town'][825:830])
print("\n")

# NaN값 바로 앞의 값으로 변경
df['embark_town'].fillna(method='ffill', inplace=True)
print(df['embark_town'][825:830])

825     Queenstown
826    Southampton
827      Cherbourg
828     Queenstown
829            NaN
Name: embark_town, dtype: object


825     Queenstown
826    Southampton
827      Cherbourg
828     Queenstown
829     Queenstown
Name: embark_town, dtype: object


### 2. 중복 데이터 처리

#### 중복 데이터 확인
- duplicated() 메소드 이용
- 중복되는 행인 경우 True, 아니면 False 반환

In [17]:
import pandas as pd
df = pd.DataFrame({'c1':['a','a','b','a','b'],
                  'c2':[1,1,1,2,2],
                  'c3':[1,1,2,2,2]})
print(df)
print('\n')

# 중복값 찾기
df_dup = df.duplicated()
print(df_dup)

  c1  c2  c3
0  a   1   1
1  a   1   1
2  b   1   2
3  a   2   2
4  b   2   2


0    False
1     True
2    False
3    False
4    False
dtype: bool


In [18]:
# 데이터프레임의 특정 열 데이터에서 중복값 찾기
col_dup = df['c2'].duplicated()
col_dup

0    False
1     True
2     True
3    False
4     True
Name: c2, dtype: bool

#### 중복 데이터 제거
- drop_duplicates()
    - 중복되는 행 제거 후 고유한 관측값 가진 행만 남김


In [19]:
print(df)
print('\n')

df2 = df.drop_duplicates()
print(df2)

  c1  c2  c3
0  a   1   1
1  a   1   1
2  b   1   2
3  a   2   2
4  b   2   2


  c1  c2  c3
0  a   1   1
2  b   1   2
3  a   2   2
4  b   2   2


In [20]:
# 중복 여부 판별 시 subset 옵션에 해당하는 열 기준으로 판단
df3 = df.drop_duplicates(subset=['c2','c3'])
df3

,c1,c2,c3
0,a,1,1
2,b,1,2
3,a,2,2


### 3. 데이터 표준화

#### 3-1. 단위 환산
- 전체 데이터 일관성 유지

In [12]:
import pandas as pd

df = pd.read_csv('./data/auto-mpg.csv', header = None)

df.columns = ['mpg','cylinders', 'displacement', 'horsepower','weight',
             'acceleration', 'model year', 'origin', 'name']
print(df.head(3))
print("\n")

# mpg(mile per gallon) -> kpl(kilometer per liter) 변환
mpg_to_kpl = 1.60934/3.78541

# kpl 열 추가
df['kpl'] = df['mpg'] * mpg_to_kpl
print(df.head(3))
print('\n')

# 소수점 둘째자리에서 반올림
df['kpl'] = df['kpl'].round(2)
print(df.head(3))

    mpg  cylinders  displacement horsepower  weight  acceleration  model year  \
0  18.0          8         307.0      130.0  3504.0          12.0          70   
1  15.0          8         350.0      165.0  3693.0          11.5          70   
2  18.0          8         318.0      150.0  3436.0          11.0          70   

   origin                       name  
0       1  chevrolet chevelle malibu  
1       1          buick skylark 320  
2       1         plymouth satellite  


    mpg  cylinders  displacement horsepower  weight  acceleration  model year  \
0  18.0          8         307.0      130.0  3504.0          12.0          70   
1  15.0          8         350.0      165.0  3693.0          11.5          70   
2  18.0          8         318.0      150.0  3436.0          11.0          70   

   origin                       name       kpl  
0       1  chevrolet chevelle malibu  7.652571  
1       1          buick skylark 320  6.377143  
2       1         plymouth satellite  7.65257

#### 3-2. 자료형 변환
- info() 함수로 각 열의 자료형 확인 가능

In [4]:
df.dtypes

mpg             float64
cylinders         int64
displacement    float64
horsepower       object
weight          float64
acceleration    float64
model year        int64
origin            int64
name             object
kpl             float64
dtype: object

In [15]:
# horsepower -> 숫자형으로 변환

# 고유값 확인
df['horsepower'].unique()

# 문자열 ? -> NaN 값으로 변환
import numpy as np
df['horsepower'].replace('?', np.nan, inplace = True)
# 누락 데이터 삭제
df.dropna(subset=['horsepower'], axis = 0, inplace = True)
# 실수형으로 변환
df['horsepower'] = df['horsepower'].astype('float')

# 자료형 확인
df['horsepower'].dtypes

dtype('float64')

In [16]:
# origin열 정수형 데이터 -> 실제 국가 이름으로 변환
print(df['origin'].unique())

df['origin'].replace({1:'USA',2:'EU',3:'JPN'}, inplace = True)

print(df['origin'].unique())
print(df['origin'].dtypes)

[1 3 2]
['USA' 'JPN' 'EU']
object


In [17]:
# 유한개의 고유값이 반복적으로 나타나는 경우 -> 범주형으로 표현
# 문자열 -> 범주형
df['origin'] = df['origin'].astype('category')
print(df['origin'].dtypes)
# 범주형 -> 문자열
df['origin'] = df['origin'].astype('str')
print(df['origin'].dtypes)

category
object


In [18]:
# model year 열의 정수형을 범주형으로 변환
print(df['model year'].sample(3)) # 무작위 3개 출력
df['model year'] = df['model year'].astype('category')
print(df['model year'].sample(3))

14     70
127    74
57     72
Name: model year, dtype: category
Categories (13, int64): [70, 71, 72, 73, ..., 79, 80, 81, 82]
230    77
61     72
60     72
Name: model year, dtype: category
Categories (13, int64): [70, 71, 72, 73, ..., 79, 80, 81, 82]


### 4. 범주형 데이터 처리

#### 4-1. 구간 분할
- 연속 데이터를 일정한 구간(bin)으로 나눠 분석하는 것이 효율적인 경우
- 구간 분할(binning) : 연속 변수 일정 구간으로 나누고 각 구간 범주형 이산 변수로 변환

In [21]:
count, bin_dividers = np.histogram(df['horsepower'], bins = 3)
print(bin_dividers)

bin_names =['저출력','보통출력','고출력']

df['hp_bin'] = pd.cut(x=df['horsepower'], # 데이터 배열
                     bins = bin_dividers, # 경계값 리스트
                     labels= bin_names, # bin 이름
                      include_lowest = True) # 첫 경계값 포함
df[['horsepower','hp_bin']].head(15)

[ 46.         107.33333333 168.66666667 230.        ]


,horsepower,hp_bin
0,130.0,보통출력
1,165.0,보통출력
2,150.0,보통출력
3,150.0,보통출력
4,140.0,보통출력
5,198.0,고출력
6,220.0,고출력
7,215.0,고출력
8,225.0,고출력
9,190.0,고출력


#### 4-2. 더미 변수
- 범주형 데이터를 회귀분석 등 머신러닝 알고리즘에 바로 사용할 수 없는 경우 -> 컴퓨터가 인식 가능한 값으로 변환 필요
- 더미변수 : 0 또는 1로 표현 / 어떤 특성의 존재 여부 표시
- 원핫벡터로 변환 -> 원핫인코딩
- get_dummies() 함수 사용

In [22]:
# 판다스 get_dummies 활용
horsepower_dummies = pd.get_dummies(df['hp_bin'])
print(horsepower_dummies.head(15))

    저출력  보통출력  고출력
0     0     1    0
1     0     1    0
2     0     1    0
3     0     1    0
4     0     1    0
5     0     0    1
6     0     0    1
7     0     0    1
8     0     0    1
9     0     0    1
10    0     0    1
11    0     1    0
12    0     1    0
13    0     0    1
14    1     0    0


In [24]:
# sklearn 라이브러리 활용
from sklearn import preprocessing

# 전처리를 위한 encoder 객체 만들기
label_encoder = preprocessing.LabelEncoder()
onehot_encoder = preprocessing.OneHotEncoder()

# label encoder로 문자열 범주 -> 숫자형 범주로 변환
onehot_labeled = label_encoder.fit_transform(df['hp_bin'].head(15))
print(onehot_labeled)
print(type(onehot_labeled))

# 2차원 행렬로 변환
onehot_reshaped = onehot_labeled.reshape(len(onehot_labeled),1)
print(onehot_reshaped)
print(type(onehot_reshaped))

# 최소행렬로 변환
onehot_fitted = onehot_encoder.fit_transform(onehot_reshaped)
print(onehot_fitted)
print(type(onehot_fitted))

[1 1 1 1 1 0 0 0 0 0 0 1 1 0 2]
<class 'numpy.ndarray'>
[[1]
 [1]
 [1]
 [1]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [1]
 [0]
 [2]]
<class 'numpy.ndarray'>
  (0, 1)	1.0
  (1, 1)	1.0
  (2, 1)	1.0
  (3, 1)	1.0
  (4, 1)	1.0
  (5, 0)	1.0
  (6, 0)	1.0
  (7, 0)	1.0
  (8, 0)	1.0
  (9, 0)	1.0
  (10, 0)	1.0
  (11, 1)	1.0
  (12, 1)	1.0
  (13, 0)	1.0
  (14, 2)	1.0
<class 'scipy.sparse.csr.csr_matrix'>


### 5. 정규화
- 각 숫자데이터의 상대적 크기 차이로 인해 머신러닝 분석 결과 달라질 수 있음.
- 상대적으로 큰 값을 갖는 변수의 영향이 더 큼
- 정규화 : 각 변수에 속하는 데이터값을 동일 크기 기준으로 나눈 비율로 나타내는 것

In [26]:
# 최대값으로 나누기
print(df.horsepower.describe())
print("\n")

df.horsepower = df.horsepower/abs(df.horsepower.max())

print(df.horsepower.head())
print("\n")
print(df.horsepower.describe())

count    392.000000
mean       0.454215
std        0.167353
min        0.200000
25%        0.326087
50%        0.406522
75%        0.547826
max        1.000000
Name: horsepower, dtype: float64


0    0.565217
1    0.717391
2    0.652174
3    0.652174
4    0.608696
Name: horsepower, dtype: float64


count    392.000000
mean       0.454215
std        0.167353
min        0.200000
25%        0.326087
50%        0.406522
75%        0.547826
max        1.000000
Name: horsepower, dtype: float64


In [27]:
# 최대값, 최소값 뺀 값으로 나누기
# 분자 : 각 열 데이터 - 최소값
# 분모 : 최대값 - 최소값

min_x = df.horsepower - df.horsepower.min()
min_max = df.horsepower.max() - df.horsepower.min()
df.horsepower = min_x/min_max

print(df.horsepower.head())
print("\n")
print(df.horsepower.describe())

0    0.456522
1    0.646739
2    0.565217
3    0.565217
4    0.510870
Name: horsepower, dtype: float64


count    392.000000
mean       0.317768
std        0.209191
min        0.000000
25%        0.157609
50%        0.258152
75%        0.434783
max        1.000000
Name: horsepower, dtype: float64


### 6. 시계열 데이터
- Timestamp : 특정 시점 기록
- Period : 두 시점 사이 일정 기간 

#### 6-1. 다른 자료형을 시계열 객체로 변환
- 판다스 to_datetime()
    - 다른 자료형 -> 판다스 Timestamp로 변환
    - datetime64 자료형

In [28]:
df = pd.read_csv('./data/stock-data.csv')

print(df.head())
print(df.info())

         Date  Close  Start   High    Low  Volume
0  2018-07-02  10100  10850  10900  10000  137977
1  2018-06-29  10700  10550  10900   9990  170253
2  2018-06-28  10400  10900  10950  10150  155769
3  2018-06-27  10900  10800  11050  10500  133548
4  2018-06-26  10800  10900  11000  10700   63039
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Date    20 non-null     object
 1   Close   20 non-null     int64 
 2   Start   20 non-null     int64 
 3   High    20 non-null     int64 
 4   Low     20 non-null     int64 
 5   Volume  20 non-null     int64 
dtypes: int64(5), object(1)
memory usage: 1.1+ KB
None


In [29]:
# 'Date'열의 날짜데이터 -> datetime64 자료형으로 변환
df['new_Date'] = pd.to_datetime(df['Date'])

print(df.head())
print("\n")
print(df.info())
print("\n")
print(type(df['new_Date'][0]))

         Date  Close  Start   High    Low  Volume   new_Date
0  2018-07-02  10100  10850  10900  10000  137977 2018-07-02
1  2018-06-29  10700  10550  10900   9990  170253 2018-06-29
2  2018-06-28  10400  10900  10950  10150  155769 2018-06-28
3  2018-06-27  10900  10800  11050  10500  133548 2018-06-27
4  2018-06-26  10800  10900  11000  10700   63039 2018-06-26


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   Date      20 non-null     object        
 1   Close     20 non-null     int64         
 2   Start     20 non-null     int64         
 3   High      20 non-null     int64         
 4   Low       20 non-null     int64         
 5   Volume    20 non-null     int64         
 6   new_Date  20 non-null     datetime64[ns]
dtypes: datetime64[ns](1), int64(5), object(1)
memory usage: 1.2+ KB
None


<class 'pandas._libs.tslibs.timestamps.T

In [30]:
# 시계열 값으로 변환된 열 새로운 행 인덱스로 지정
# DatetimeIndex로 저장 -> 시간 순서에 맞춰 인덱싱/슬라이싱 편리
df.set_index('new_Date', inplace= True)
df.drop('Date', axis=1, inplace = True)

print(df.head())
print("\n")
print(df.info())

            Close  Start   High    Low  Volume
new_Date                                      
2018-07-02  10100  10850  10900  10000  137977
2018-06-29  10700  10550  10900   9990  170253
2018-06-28  10400  10900  10950  10150  155769
2018-06-27  10900  10800  11050  10500  133548
2018-06-26  10800  10900  11000  10700   63039


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 20 entries, 2018-07-02 to 2018-06-01
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   Close   20 non-null     int64
 1   Start   20 non-null     int64
 2   High    20 non-null     int64
 3   Low     20 non-null     int64
 4   Volume  20 non-null     int64
dtypes: int64(5)
memory usage: 960.0 bytes
None


In [31]:
# Timestamp -> Period로 변환
dates = ['2019-01-01', '2020-03-01', '2021-06-01']

ts_dates = pd.to_datetime(dates)
print(ts_dates)
print("\n")

pr_day = ts_dates.to_period(freq='D') # 1일
print(pr_day)
pr_month = ts_dates.to_period(freq='M') # 1개월
print(pr_month)
pr_year = ts_dates.to_period(freq='A') # 1년(12월기준)
print(pr_year)

DatetimeIndex(['2019-01-01', '2020-03-01', '2021-06-01'], dtype='datetime64[ns]', freq=None)


PeriodIndex(['2019-01-01', '2020-03-01', '2021-06-01'], dtype='period[D]', freq='D')
PeriodIndex(['2019-01', '2020-03', '2021-06'], dtype='period[M]', freq='M')
PeriodIndex(['2019', '2020', '2021'], dtype='period[A-DEC]', freq='A-DEC')


#### 6-2. 시계열 데이터 만들기
- date_range() 함수 사용

In [32]:
# 월 간격, 월 시작일 기준
ts_ms = pd.date_range(start='2019-01-01', # 시작
                     end = None, # 끝
                     periods = 6, # 생성할 Timestamp 개수
                     freq='MS', # 시간 간격(MS: 월 시작일)
                     tz='Asia/Seoul') # 시간대
print(ts_ms)

DatetimeIndex(['2019-01-01 00:00:00+09:00', '2019-02-01 00:00:00+09:00',
               '2019-03-01 00:00:00+09:00', '2019-04-01 00:00:00+09:00',
               '2019-05-01 00:00:00+09:00', '2019-06-01 00:00:00+09:00'],
              dtype='datetime64[ns, Asia/Seoul]', freq='MS')


In [33]:
# 월 간격, 월 마지막 날 기준
ts_me = pd.date_range(start='2019-01-01', # 시작
                     periods = 6, # 생성할 Timestamp 개수
                     freq='M', # 시간 간격(MS: 월 시작일)
                     tz='Asia/Seoul') # 시간대
print(ts_me)
print("\n")

# 분기(3개월) 간격, 월 마지막 날 기준
ts_3m = pd.date_range(start='2019-01-01', # 시작
                     periods = 6, # 생성할 Timestamp 개수
                     freq='3M', # 시간 간격(MS: 월 시작일)
                     tz='Asia/Seoul') # 시간대
print(ts_3m)

DatetimeIndex(['2019-01-31 00:00:00+09:00', '2019-02-28 00:00:00+09:00',
               '2019-03-31 00:00:00+09:00', '2019-04-30 00:00:00+09:00',
               '2019-05-31 00:00:00+09:00', '2019-06-30 00:00:00+09:00'],
              dtype='datetime64[ns, Asia/Seoul]', freq='M')


DatetimeIndex(['2019-01-31 00:00:00+09:00', '2019-04-30 00:00:00+09:00',
               '2019-07-31 00:00:00+09:00', '2019-10-31 00:00:00+09:00',
               '2020-01-31 00:00:00+09:00', '2020-04-30 00:00:00+09:00'],
              dtype='datetime64[ns, Asia/Seoul]', freq='3M')


In [35]:
# Period 배열 - 1개월 길이
pr_m = pd.period_range(start='2019-01-01',
                      end=None,
                      periods=3,
                      freq='M')
print(pr_m)

PeriodIndex(['2019-01', '2019-02', '2019-03'], dtype='period[M]', freq='M')


In [36]:
# Period 배열 - 1시간 길이
pr_h = pd.period_range(start='2019-01-01',
                      end=None,
                      periods=3,
                      freq='H')
print(pr_h)
print("\n")

# Period 배열 - 2시간 길이
pr_2h = pd.period_range(start='2019-01-01',
                      end=None,
                      periods=3,
                      freq='2H')
print(pr_2h)

PeriodIndex(['2019-01-01 00:00', '2019-01-01 01:00', '2019-01-01 02:00'], dtype='period[H]', freq='H')


PeriodIndex(['2019-01-01 00:00', '2019-01-01 02:00', '2019-01-01 04:00'], dtype='period[2H]', freq='2H')


#### 6-3. 시계열 데이터 활용

In [39]:
# 날짜 데이터 분리

df = pd.read_csv('./data/stock-data.csv')

df['new_Date'] = pd.to_datetime(df['Date'])
print(df.head())
print("\n")

df["Year"] = df['new_Date'].dt.year
df['Month'] = df['new_Date'].dt.month
df['Day'] = df['new_Date'].dt.day
print(df.head())

         Date  Close  Start   High    Low  Volume   new_Date
0  2018-07-02  10100  10850  10900  10000  137977 2018-07-02
1  2018-06-29  10700  10550  10900   9990  170253 2018-06-29
2  2018-06-28  10400  10900  10950  10150  155769 2018-06-28
3  2018-06-27  10900  10800  11050  10500  133548 2018-06-27
4  2018-06-26  10800  10900  11000  10700   63039 2018-06-26


         Date  Close  Start   High    Low  Volume   new_Date  Year  Month  Day
0  2018-07-02  10100  10850  10900  10000  137977 2018-07-02  2018      7    2
1  2018-06-29  10700  10550  10900   9990  170253 2018-06-29  2018      6   29
2  2018-06-28  10400  10900  10950  10150  155769 2018-06-28  2018      6   28
3  2018-06-27  10900  10800  11050  10500  133548 2018-06-27  2018      6   27
4  2018-06-26  10800  10900  11000  10700   63039 2018-06-26  2018      6   26


In [41]:
# Timestamp -> Period로 변환
df['Date_yr'] = df['new_Date'].dt.to_period(freq='A')
df['Date_m'] = df['new_Date'].dt.to_period(freq='M')
print(df.head())

         Date  Close  Start   High    Low  Volume   new_Date  Year  Month  \
0  2018-07-02  10100  10850  10900  10000  137977 2018-07-02  2018      7   
1  2018-06-29  10700  10550  10900   9990  170253 2018-06-29  2018      6   
2  2018-06-28  10400  10900  10950  10150  155769 2018-06-28  2018      6   
3  2018-06-27  10900  10800  11050  10500  133548 2018-06-27  2018      6   
4  2018-06-26  10800  10900  11000  10700   63039 2018-06-26  2018      6   

   Day Date_yr   Date_m  
0    2    2018  2018-07  
1   29    2018  2018-06  
2   28    2018  2018-06  
3   27    2018  2018-06  
4   26    2018  2018-06  


In [42]:
# 추출한 날짜 정보 행 인덱스로 지정
df.set_index('Date_m', inplace=True)
print(df.head())

               Date  Close  Start   High    Low  Volume   new_Date  Year  \
Date_m                                                                     
2018-07  2018-07-02  10100  10850  10900  10000  137977 2018-07-02  2018   
2018-06  2018-06-29  10700  10550  10900   9990  170253 2018-06-29  2018   
2018-06  2018-06-28  10400  10900  10950  10150  155769 2018-06-28  2018   
2018-06  2018-06-27  10900  10800  11050  10500  133548 2018-06-27  2018   
2018-06  2018-06-26  10800  10900  11000  10700   63039 2018-06-26  2018   

         Month  Day Date_yr  
Date_m                       
2018-07      7    2    2018  
2018-06      6   29    2018  
2018-06      6   28    2018  
2018-06      6   27    2018  
2018-06      6   26    2018  


In [43]:
# 날짜 인덱스 활용
# Period 구성 열 행 인덱스 지정 -> PeriodIndex

df = pd.read_csv('./data/stock-data.csv')

df['new_Date'] = pd.to_datetime(df['Date'])
df.set_index('new_Date', inplace=True)
print(df.head())
print("\n")
print(df.index)

                  Date  Close  Start   High    Low  Volume
new_Date                                                  
2018-07-02  2018-07-02  10100  10850  10900  10000  137977
2018-06-29  2018-06-29  10700  10550  10900   9990  170253
2018-06-28  2018-06-28  10400  10900  10950  10150  155769
2018-06-27  2018-06-27  10900  10800  11050  10500  133548
2018-06-26  2018-06-26  10800  10900  11000  10700   63039


DatetimeIndex(['2018-07-02', '2018-06-29', '2018-06-28', '2018-06-27',
               '2018-06-26', '2018-06-25', '2018-06-22', '2018-06-21',
               '2018-06-20', '2018-06-19', '2018-06-18', '2018-06-15',
               '2018-06-14', '2018-06-12', '2018-06-11', '2018-06-08',
               '2018-06-07', '2018-06-05', '2018-06-04', '2018-06-01'],
              dtype='datetime64[ns]', name='new_Date', freq=None)


In [46]:
# 날짜 인덱스로 인덱싱/슬라이싱
df_y = df.loc['2018']
print(df_y.head())
print("\n")
df_ym = df.loc['2018-07']
print(df_ym)
print("\n")
df_ym_cols= df.loc['2018-07','Start':'High']
print(df_ym_cols)
print("\n")
df_ymd = df.loc['2018-07-02']
print(df_ymd)
print("\n")
df_ymd_range = df['2018-06-25':'2018-06-20']
print(df_ymd_range)


                  Date  Close  Start   High    Low  Volume
new_Date                                                  
2018-07-02  2018-07-02  10100  10850  10900  10000  137977
2018-06-29  2018-06-29  10700  10550  10900   9990  170253
2018-06-28  2018-06-28  10400  10900  10950  10150  155769
2018-06-27  2018-06-27  10900  10800  11050  10500  133548
2018-06-26  2018-06-26  10800  10900  11000  10700   63039


                  Date  Close  Start   High    Low  Volume
new_Date                                                  
2018-07-02  2018-07-02  10100  10850  10900  10000  137977


            Start   High
new_Date                
2018-07-02  10850  10900


                  Date  Close  Start   High    Low  Volume
new_Date                                                  
2018-07-02  2018-07-02  10100  10850  10900  10000  137977


                  Date  Close  Start   High    Low  Volume
new_Date                                                  
2018-06-25  2018-06-25  11150  1

In [47]:
# 시간 간격 계산
today = pd.to_datetime('2018-12-25') # 기준일 생성
df['time_delta'] = today - df.index
df.set_index('time_delta', inplace=True)
df_180 = df['180 days':'189 days']
print(df_180)

                  Date  Close  Start   High    Low  Volume
time_delta                                                
180 days    2018-06-28  10400  10900  10950  10150  155769
181 days    2018-06-27  10900  10800  11050  10500  133548
182 days    2018-06-26  10800  10900  11000  10700   63039
183 days    2018-06-25  11150  11400  11450  11000   55519
186 days    2018-06-22  11300  11250  11450  10750  134805
187 days    2018-06-21  11200  11350  11750  11200  133002
188 days    2018-06-20  11550  11200  11600  10900  308596
189 days    2018-06-19  11300  11850  11950  11300  180656
